## Data Preparation & Cleaning

This notebook focuses on transforming raw food delivery and external datasets into a clean, reliable, and analysis-ready format.

The objective of this step is to ensure data quality, consistency, and proper alignment across time and location before performing exploratory analysis or predictive modeling. All cleaning decisions are made with business interpretability and real-world applicability in mind

In [1]:
# Loading Required Libraries
import pandas as pd
import numpy as np

### Loading Raw Datasets

In this step, all raw datasets are loaded separately to understand their structure, schema, and data quality.

The datasets include:
- Food delivery order data (core business dataset)
- Daily weather data
- Fuel price data
- Indian holiday data

Loading datasets individually helps in identifying inconsistencies early and prevents errors during integration.

In [2]:
# All file paths are defined relative to the notebook directory to ensure portability across environments.
from pathlib import Path

BASE_DIR = Path("..")
DATA_DIR = BASE_DIR / "data"

RAW_DIR = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"

FUEL_DIR = RAW_DIR / "fuel"
HOLIDAYS_DIR = RAW_DIR / "indian_holidays"

# Loading Datasets
orders_df = pd.read_csv(RAW_DIR / "order_history.csv")
weather_df = pd.read_csv(RAW_DIR / "weather_daily.csv")
petrol_df = pd.read_csv(FUEL_DIR / "petrol.csv")
diesel_df = pd.read_csv(FUEL_DIR / "diesel.csv")
holidays_df = pd.read_csv(HOLIDAYS_DIR / "2018.csv")

### Column Name Standardization

To improve consistency and reduce the risk of errors in below analysis and modeling,
all column names are standardized by converting them to lowercase and replacing spaces
with underscores.

In [3]:
orders_df.columns = (
    orders_df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace(r"[()]", "", regex=True)
)

### Initial Data Inspection

Before applying any cleaning logic, it is important to understand:
- Number of records
- Available columns
- Data types
- Presence of missing or inconsistent values

This step ensures transparency and helps define a justified cleaning strategy.

In [4]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21321 entries, 0 to 21320
Data columns (total 29 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   restaurant_id                                     21321 non-null  int64  
 1   restaurant_name                                   21321 non-null  object 
 2   subzone                                           21321 non-null  object 
 3   city                                              21321 non-null  object 
 4   order_id                                          21321 non-null  int64  
 5   order_placed_at                                   21321 non-null  object 
 6   order_status                                      21321 non-null  object 
 7   delivery                                          21321 non-null  object 
 8   distance                                          21321 non-null  object 
 9   items_in_order   

### Date and Time Processing

Order timestamps are converted into a standardized datetime format.
A derived order date column is created to support daily-level aggregation and time-series analysis.

In [5]:
orders_df['order_placed_at'] = pd.to_datetime(orders_df['order_placed_at'],errors='coerce')
orders_df['order_date'] = orders_df['order_placed_at'].dt.date

C:\Users\lenovo\AppData\Local\Temp\ipykernel_18976\2235490896.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  orders_df['order_placed_at'] = pd.to_datetime(orders_df['order_placed_at'],errors='coerce')


### Filtering Valid Orders

Only successfully delivered orders are retained for demand and revenue analysis.

In [6]:
orders_df = orders_df[orders_df['order_status'] == 'Delivered']

### Location Standardization (City & Subzone)

City and subzone fields are cleaned to remove inconsistencies such as
extra spaces and mixed casing. Standardizing location fields.

In [7]:
orders_df['city'] = orders_df['city'].str.strip().str.title()
orders_df['subzone'] = orders_df['subzone'].str.strip().str.title()

### Numeric Field Validation and Conversion

In [8]:
numeric_cols = [
    'bill_subtotal',
    'packaging_charges',
    'restaurant_discount_promo',
    'restaurant_discount_flat_offs,_freebies_&_others',
    'gold_discount',
    'brand_pack_discount',
    'total',
    'kpt_duration_minutes',
    'rider_wait_time_minutes'
]

for col in numeric_cols:
    orders_df[col] = pd.to_numeric(orders_df[col], errors='coerce')

In [9]:
orders_df[numeric_cols].describe()

,bill_subtotal,packaging_charges,restaurant_discount_promo,"restaurant_discount_flat_offs,_freebies_&_others",gold_discount,brand_pack_discount,total,kpt_duration_minutes,rider_wait_time_minutes
count,21131.000000,21131.000000,21131.000000,21131.000000,21131.000000,21131.000000,21131.000000,20934.000000,21082.000000
mean,749.955372,32.563412,65.046085,31.758953,0.097369,3.046712,682.569673,17.339428,4.826174
std,496.534592,22.154775,84.688792,131.575999,3.256260,17.105073,463.617299,6.277504,4.982873
min,50.000000,0.000000,0.000000,0.000000,0.000000,0.000000,52.500000,0.000000,0.100000
25%,459.000000,18.460000,0.000000,0.000000,0.000000,0.000000,388.500000,13.380000,1.000000
50%,629.000000,28.450000,80.000000,0.000000,0.000000,0.000000,597.450000,16.350000,3.100000
75%,899.000000,39.950000,100.000000,0.000000,0.000000,0.000000,837.925000,20.050000,7.400000
max,16080.000000,603.000000,4020.000000,7787.000000,280.100000,554.800000,12663.000000,90.870000,73.800000


## Distance and Item Count Processing

Distance and item count fields are converted from text-based representations
into numeric formats to enable quantitative analysis of delivery effort
and order size.


In [10]:
orders_df['distance_km'] = (
    orders_df['distance']
    .str.extract(r'([\d\.]+)')
    .astype(float)
)

orders_df['items_count'] = (
    orders_df['items_in_order']
    .str.extract(r'(\d+)')
    .astype(float)
)


In [11]:
orders_df[['distance_km', 'items_count']].describe()

,distance_km,items_count
count,21131.000000,21131.000000
mean,4.176092,1.091903
std,2.969507,0.385708
min,1.000000,1.000000
25%,2.000000,1.000000
50%,3.000000,1.000000
75%,6.000000,1.000000
max,21.000000,26.000000


### Missing Value Handling Strategy

Dropping critical fields with missing values makes data more realiable. Replacing Missing values with median makes a handling of outliers more robust.

In [12]:
# dropping critical fields with missing values
orders_df = orders_df.dropna(subset=['order_date', 'city', 'total'])

In [13]:
orders_df['kpt_duration_minutes'] = orders_df['kpt_duration_minutes'].fillna(orders_df['kpt_duration_minutes'].median())

orders_df['rider_wait_time_minutes'] = orders_df['rider_wait_time_minutes'].fillna(orders_df['rider_wait_time_minutes'].median())

In [14]:
orders_df.isnull().sum().sort_values(ascending=False).head(10)

restaurant_penalty_rejection            21131
cancellation_/_rejection_reason         21131
restaurant_compensation_cancellation    21131
review                                  20835
customer_complaint_tag                  20662
instructions                            20422
rating                                  18640
discount_construct                       5431
restaurant_id                               0
kpt_duration_minutes                        0
dtype: int64

### Removing Low-Signal and Sparse Columns

Several columns contain extremely high missing values or unstructured text
that do not contribute meaningfully to demand forecasting, revenue analysis,
or operational modeling.

These columns are removed to reduce noise, simplify the dataset, and
improve model stability.


In [15]:
columns_to_drop = [
    'restaurant_penalty_rejection',
    'restaurant_compensation_cancellation',
    'cancellation_/_rejection_reason',
    'review',
    'customer_complaint_tag',
    'instructions',
    'rating',
    'discount_construct'
]

orders_df = orders_df.drop(columns=columns_to_drop, errors='ignore')


### Saving Final Cleaned Dataset

The dataset is now fully cleaned, standardized, and validated.
This version will be used for all exploratory analysis, forecasting,
and machine learning tasks.


In [16]:
orders_df.to_csv(PROCESSED_DIR / "order_history_cleaned.csv", index=False)